In [ ]:
import warnings
import pip
import site
import importlib

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    libs = ['numpy', 'pandas', 'openai']
    for lib in libs:
        pip.main(['install', '--disable-pip-version-check', '-q', lib])
    importlib.reload(site)

In [11]:
path = '/home/coder/project/QASC_Dataset/'
files = ['train.jsonl', 'dev.jsonl']

In [12]:
import pandas as pd

cols = ['formatted_question', 'combinedfact', 'answerKey']
train = pd.read_json(path + files[0], lines=True)[cols]
dev = pd.read_json(path + files[1], lines=True)[cols]

In [13]:
# merge train and dev
train = pd.concat([train, dev], ignore_index=True)

In [14]:
len(train)

9060

In [15]:
train = train.apply(lambda x: x.astype(str).str.lower())

In [24]:
from dotenv import load_dotenv
import openai
import time
import numpy as np
import os
load_dotenv()


openai.api_key = os.getenv("OPENAI_API_KEY")

class TopicMaker:
  def __init__(self, query_file, max_queries=3):
    with open(query_file, 'r') as f:
      query = f.read().lower()
    self.query = query
    self.max_queries = max_queries

  def gen_query(self, question):
    new_line = f'question: {question}\ntopics:'
    return self.query + new_line
  
  def send_query(self, query):
    response = None
    for _ in range(self.max_queries):
      try:
        response = openai.Completion.create(
          model="text-babbage-001",
          prompt=query,
          temperature=0,
          max_tokens=60,
          top_p=1.0,
          frequency_penalty=0.5,
          presence_penalty=0.0
        )
        # random sleep seconds 
        time.sleep(np.random.randint(1, 5))
        break
      except Exception as e:
        print('Error', e)
      
    return response
  
  def get_topics(self, response):
    if response is None:
      return []
    return response['choices'][0]['text'].strip().lower().split(', ')
  
  def __call__(self, question):
    query = self.gen_query(question)
    response = self.send_query(query)
    topics = self.get_topics(response)
    return topics




In [26]:
tm = TopicMaker('query.txt')
q = train['formatted_question'][0]
print('Question:', q)
print('Query:')
print(tm.gen_query(q))
print('Response:')
tm(train['formatted_question'][0])

Question: what type of water formation is formed by clouds? (a) pearls (b) streams (c) shells (d) diamonds (e) rain (f) beads (g) cooled (h) liquid
Query:
determine general topics for each question. don't add trailing comma at the end!

# example
question:  what are used for protection by fish? (a) scales (b) fins (c) streams. (d) coral (e) gills (f) collagen (g) mussels (h) whiskers
topics: biology, anatomy, marine biology, zoology, evolution, fish anatomy, aquatic ecosystems, natural selection

question:  what are pangolins covered in? (a) tunicates (b) echinoids (c) shells (d) exoskeleton (e) blastoids (f) barrel-shaped (g) protection (h) white
topics: biology, anatomy, zoology, endangered species, wildlife conservation, animal behavior, mammals

question:  what are covered with protection? (a) apples (b) trees (c) coral (d) clams (e) roses (f) wings (g) hats (h) fish
topics: biology, anatomy, botany, botanical morphology, horticulture, agriculture, plant physiology, plant reproduct

['physics', 'clouds', 'atmospheric science', 'meteorology']

In [27]:
train_truncated = train[:3]

# train_truncated['topics'] = train_truncated['formatted_question'].apply(tm)

C:\Users\rodio\AppData\Local\Temp\ipykernel_11228\3282831780.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_truncated['topics'] = train_truncated['formatted_question'].apply(tm)


In [28]:
train_truncated

,formatted_question,combinedfact,answerKey,topics
0,what type of water formation is formed by clou...,beads of water can be formed by clouds.,f,"[physics, clouds, atmospheric science, meteoro..."
1,where do beads of water come from? (a) too muc...,vapor turning into a liquid leaves behind bead...,e,"[geography, science, environmental science]"
2,what forms beads of water? (a) necklaces. (b)...,steam forms beads of water.,b,"[physics, engineering, chemistry, physics]"


In [30]:
import pickle

topics_file = '/output/topics.pkl'
if not os.path.exists(topics_file):
  with open(topics_file, 'wb') as f:
    pickle.dump(pd.Series(), f)

batch_size = 1000
start_point = 5000
for i in range(start_point, len(train), batch_size):
    new_topics = train[i:i+batch_size]['formatted_question'].apply(tm)
    with open(topics_file, 'rb') as f:
        topics = pickle.load(f)
    topics = pd.concat([topics, new_topics])
    with open(topics_file, 'wb') as f:
        pickle.dump(topics, f)

print('Success!')

C:\Users\rodio\AppData\Local\Temp\ipykernel_11228\2895107149.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pickle.dump(pd.Series(), f)


Error That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 525dd04aed2d2cee3b991dff2604215d in your message.)
Error Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000027FAC3C6F90>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000027FAC30E250>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: 